In [1]:
!pip install icecream

In [16]:
from io import StringIO #Extract
import requests #Extract
from bs4 import BeautifulSoup #Extract
import pandas as pd #Transformation
import sqlite3 #LOAD
from datetime import datetime
from icecream import ic

In [17]:
for i in range(1,5):
    a=200
    b=250                         # Python Version error
    c=300
    ans= ic(a)*(b)/(c)
    a-=10
    b+=20


ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?
ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?
ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?
ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?


In [18]:
# Step 0
def log_progress(message):
    with open('./logs/code_log.txt', 'a') as f:
        f.write(f'{datetime.now()}, {message}.\n')

In [44]:
# step 1 Extraction
def extract(url, table_att):
    soup = requests.get(url).text
    web = BeautifulSoup(soup, 'html.parser')
    table = web.find('span', string=table_att).findNext('table')
    df = pd.read_html(StringIO(str(table)))[0]

    log_progress('Data Extracted successfully from Wikipedia.')

    return df

In [45]:
# Step 2 Transformation
def transformation(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col = 0).to_dict()['Rate']

    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['GBP'],2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['EUR'],2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['INR'],2)

    print(df['MC_EUR_Billion'][4])

    log_progress('Data Transformation done now Step 2 completed.')
    
    return df

In [46]:
#Step 3 Load
#CSV
def load_to_csv(df, out_path):
    df.to_csv(out_path)

    log_progress('Data Loaded into the CSV File.')

In [47]:
# Step 3
#SQLite 
def load_to_db(df, sql_conn, tn):
    df.to_sql(tn, sql_conn, if_exists= 'replace', index= False)

    log_progress('Data Loaded into the SQLite Database')

In [48]:
if __name__ =='__main__':
    url= 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    
    
    log_progress ('Preliminaries complete. Initiating ETL process')

    df=extract(url, 'by market capatilization')
    
    print(df)

    transformation(df, output_csv_path)
    


ConnectionError: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000227ACFEDDD0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))